In [1]:
# Set the range for mixing parameters
import numpy as np

step_total  = 18
step_size = 0.05
MU = np.around(np.arange(step_size, step_size * step_total + 0.01, step_size), decimals=2)


# Specify Parameters
number_of_comm = 15           # Number of communities
comm_size = 68  # Size of communities
n = number_of_comm*comm_size          # Total number of nodes
deg_avg = 25  # Average degree of the whole network

sample_count = 20

In [2]:
import numpy as np
import networkx as nx

def generate_ppm(q, comm_size, deg_avg, mu):
    n = q*comm_size # total number of nodes
    # Calculate p_out and p_in
    p_out = mu*deg_avg/n
    p_in = (deg_avg - (n - n / q) * p_out) / (n / q - 1)
    #print(f'p_out={p_out}, p_in={p_in}')
    if p_out < 0 or p_out > 1 or p_in < 0 or p_in > 1:
        raise ValueError("Calculated probability is out of the valid range (0, 1).")
    sizes = [n // q] * q  

    # Probability matrix (within and between blocks)
    p_matrix = [[p_in if i == j else p_out for j in range(q)] for i in range(q)]

    # Generate the PPM graph using the SBM generator
    G = nx.stochastic_block_model(sizes, p_matrix, seed=42)
    return G

In [3]:
import os
import pickle
import networkx as nx
#from networkx.generators.community import LFR_benchmark_graph  # Replace with your actual import

def gene_ppm_graph_mu(mu, sample_count=sample_count, output_dir='data'): ##########################################
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    graphs = []
    memberships = []

    for i in range(sample_count):
        G = generate_ppm(number_of_comm, comm_size, deg_avg, mu)

        G = nx.Graph(G)
        selfloop_edges = list(nx.selfloop_edges(G))
        G.remove_edges_from(selfloop_edges)

        intrinsic_membership = list(nx.get_node_attributes(G, 'block').values())

        # Append the graph and membership to the lists
        graphs.append(G)
        memberships.append(intrinsic_membership)

    # Save all graphs and memberships into a single file
    combined_data = {
        'graphs': graphs,
        'memberships': memberships
    }
    
    file_path = os.path.join(output_dir, f'ppm_graph_mu{mu}.pickle')
    with open(file_path, 'wb') as file:
        pickle.dump(combined_data, file)

    print(f'Saved all graphs and memberships for mu={mu} to {file_path}')

In [4]:
from concurrent.futures import ProcessPoolExecutor

with ProcessPoolExecutor() as executor:
    results = list(executor.map(gene_ppm_graph_mu, MU))

Saved all graphs and memberships for mu=0.05 to data/ppm_graph_mu0.05.pickle
Saved all graphs and memberships for mu=0.1 to data/ppm_graph_mu0.1.pickle
Saved all graphs and memberships for mu=0.15 to data/ppm_graph_mu0.15.pickle
Saved all graphs and memberships for mu=0.2 to data/ppm_graph_mu0.2.pickle
Saved all graphs and memberships for mu=0.4 to data/ppm_graph_mu0.4.pickleSaved all graphs and memberships for mu=0.35 to data/ppm_graph_mu0.35.pickleSaved all graphs and memberships for mu=0.5 to data/ppm_graph_mu0.5.pickleSaved all graphs and memberships for mu=0.3 to data/ppm_graph_mu0.3.pickle



Saved all graphs and memberships for mu=0.65 to data/ppm_graph_mu0.65.pickle
Saved all graphs and memberships for mu=0.25 to data/ppm_graph_mu0.25.pickle
Saved all graphs and memberships for mu=0.7 to data/ppm_graph_mu0.7.pickle
Saved all graphs and memberships for mu=0.75 to data/ppm_graph_mu0.75.pickle
Saved all graphs and memberships for mu=0.45 to data/ppm_graph_mu0.45.pickleSaved all gr

In [5]:
def load_ppm_graph_mu(mu, output_dir='data'):
    file_path = os.path.join(output_dir, f'ppm_graph_mu{mu}.pickle')
    with open(file_path, 'rb') as file:
        combined_data = pickle.load(file)
    
    graphs = combined_data['graphs']
    memberships = combined_data['memberships']
    
    return graphs, memberships

# Example usage
# graphs, memberships = load_data(mu_value)
graphs, memberships = load_ppm_graph_mu(mu=0.2)